<a href="https://colab.research.google.com/github/chathuramadusanka/Dialog_model/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/Shareddrives/chathura/dfjoint_pre.csv')

In [5]:
data

,Unnamed: 0,MSISDN_ENCR_INT,BI_ACCOUNT_ID_x,ACCOUNT_TYPE,NETWORK_STAY,GENDER,AGE,PREFERRED_LANGUAGE,DEVICE_CATEGORY,SUBSCRIPTION_PLAN,...,OVERSEAS_TRAVELLERS,ONLINE_SHOPPER,BANK_USER,ONLINE_PAYER,UPDATE_DATE,DATA_PACKAGE,BI_ACCOUNT_ID_y,VOICE_USAGE,DATA_USAGE,SMS_USAGE
0,0,-6.883145e+17,10933313,PREPAID,163,MALE,29,Tamil,Smartphone,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,Internet_card_119,10933313,0.00000,0.00000,0
1,1,-6.883145e+17,10933313,PREPAID,163,MALE,29,Tamil,Smartphone,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,Internet_card_119,10933313,13.20000,0.00000,0
2,2,-6.883145e+17,10933313,PREPAID,163,MALE,29,Tamil,Smartphone,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,Internet_card_119,10933313,3.66667,0.00000,0
3,3,-6.883145e+17,10933313,PREPAID,163,MALE,29,Tamil,Smartphone,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,Internet_card_119,10933313,13.53333,0.00000,0
4,4,-4.362307e+18,3344913228,PREPAID,0,MALE,37,Sinhala,Smartphone,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,Fun_Blaster_Plus_447,3344913228,0.00000,1921.88583,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711166,4711166,-1.784940e+18,1340721483,PREPAID,18,FEMALE,32,Sinhala,Basic,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,ComboPack_EZ_Rs469_Data,1340721483,3.18333,0.00000,0
4711167,4711167,-1.784940e+18,1340721483,PREPAID,18,FEMALE,32,Sinhala,Basic,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,ComboPack_EZ_Rs469_Data,1340721483,33.01667,0.00000,0
4711168,4711168,-1.784940e+18,1340721483,PREPAID,18,FEMALE,32,Sinhala,Basic,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,ComboPack_EZ_Rs469_Data,1340721483,48.53333,0.00228,0
4711169,4711169,-1.784940e+18,1340721483,PREPAID,18,FEMALE,32,Sinhala,Basic,PRP_16,...,0,0,0,0,2023-09-05 16:20:05.066,ComboPack_EZ_Rs469_Data,1340721483,49.88333,0.00000,0


In [6]:
data = data.drop(columns=['UPDATE_DATE','Unnamed: 0','ACCOUNT_TYPE'])

# Label encode categorical columns
label_encoder = LabelEncoder()
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])

In [7]:
data

,MSISDN_ENCR_INT,BI_ACCOUNT_ID_x,NETWORK_STAY,GENDER,AGE,PREFERRED_LANGUAGE,DEVICE_CATEGORY,SUBSCRIPTION_PLAN,VOICE_ARPU,DATA_ARPU,...,EAT_OUT_SEEKER,OVERSEAS_TRAVELLERS,ONLINE_SHOPPER,BANK_USER,ONLINE_PAYER,DATA_PACKAGE,BI_ACCOUNT_ID_y,VOICE_USAGE,DATA_USAGE,SMS_USAGE
0,-6.883145e+17,10933313,163,1,29,2,3,7,0,0,...,0,0,0,0,0,50,10933313,0.00000,0.00000,0
1,-6.883145e+17,10933313,163,1,29,2,3,7,0,0,...,0,0,0,0,0,50,10933313,13.20000,0.00000,0
2,-6.883145e+17,10933313,163,1,29,2,3,7,0,0,...,0,0,0,0,0,50,10933313,3.66667,0.00000,0
3,-6.883145e+17,10933313,163,1,29,2,3,7,0,0,...,0,0,0,0,0,50,10933313,13.53333,0.00000,0
4,-4.362307e+18,3344913228,0,1,37,1,3,7,3,308,...,0,0,0,0,0,48,3344913228,0.00000,1921.88583,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4711166,-1.784940e+18,1340721483,18,0,32,1,0,7,161,0,...,0,0,0,0,0,4,1340721483,3.18333,0.00000,0
4711167,-1.784940e+18,1340721483,18,0,32,1,0,7,161,0,...,0,0,0,0,0,4,1340721483,33.01667,0.00000,0
4711168,-1.784940e+18,1340721483,18,0,32,1,0,7,161,0,...,0,0,0,0,0,4,1340721483,48.53333,0.00228,0
4711169,-1.784940e+18,1340721483,18,0,32,1,0,7,161,0,...,0,0,0,0,0,4,1340721483,49.88333,0.00000,0


In [8]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [9]:
data_scaled

array([[-0.09269277, -1.43146278,  2.41409933, ..., -0.416339  ,
        -0.83170233, -0.08041975],
       [-0.09269277, -1.43146278,  2.41409933, ...,  0.15928912,
        -0.83170233, -0.08041975],
       [-0.09269277, -1.43146278,  2.41409933, ..., -0.25644215,
        -0.83170233, -0.08041975],
       ...,
       [-0.29816208,  0.20425165, -0.62353574, ...,  1.70011171,
        -0.83169954, -0.08041975],
       [-0.29816208,  0.20425165, -0.62353574, ...,  1.75898277,
        -0.83170233, -0.08041975],
       [-0.29816208,  0.20425165, -0.62353574, ...,  2.09621974,
        -0.83170013, -0.08041975]])

In [ ]:
eps = 0.5  # Adjust this value based on your dataset
min_samples = 5  # Adjust this value based on your dataset

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels = dbscan.fit_predict(data_scaled)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)

# Create a DataFrame with PCA components and cluster labels
clustered_data = pd.DataFrame(data={'PCA1': data_pca[:, 0], 'PCA2': data_pca[:, 1], 'Cluster': labels})


clustered_data.to_csv('clustered_data.csv', index=False)

# Create a scatter plot
plt.figure(figsize=(10, 6))
for cluster in range(-1, 5):  # Include noise points (-1) and the five clusters
    cluster_points = clustered_data[clustered_data['Cluster'] == cluster]
    plt.scatter(cluster_points['PCA1'], cluster_points['PCA2'], label=f'Cluster {cluster}')

plt.title('DBSCAN Clustering Results')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend()

# Save clustered data to a CSV file
clustered_data.to_csv('/content/drive/Shareddrives/chathura/clustered_data.csv', index=False)
plt.savefig('/content/drive/Shareddrives/chathura/dbscan_clusters.png')

plt.show()

